# Prototype Conformity loss

In [1]:
import os
import sys
sys.path.append('..')
import yaml
import shutil
import argparse
import numpy as np
from tqdm import tqdm
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader, random_split

from utils import *

## Parameter setting

In [2]:
gpu = '0,1,2,3'
dataset = 'cifar10'
model_type = 'wrn34-10'
checkpoint = './checkpoint/pc_v2/%s/%s' % (model_type, dataset)
num_classes = 10
lr = 0.01
momentum = 0.9
batch_size = 256
warmup_epochs = 200
with_pc_epochs = 300
epsilon = 8/255
alpha = 2/255
num_repeats = 10

## PC loss parameters
lr_prox = 0.5
lr_conprox = 1e-5
weight_prox = 1.0
weight_conprox = 1e-5

## Inner maximization

In [1]:
def inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats):
    noise = torch.FloatTensor(inputs.shape).uniform_(-1, 1).cuda()
    x = torch.clamp(inputs + noise, min=0, max=1)
    for _ in range(num_repeats):
        x.requires_grad_()
        logits = model(x)[0]
        loss = xent(logits, targets)
        loss.backward()
        grads = x.grad.data
        x = x.detach() + alpha*torch.sign(grads).detach()
        x = torch.min(torch.max(x, inputs - epsilon), inputs + epsilon).clamp(min=0, max=1)
    return x

## Training (Outer minimization)

In [4]:
def xent_training(epoch, model, dataloader, optimizer, num_classes):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
        logits, _ = model(x)
        
        loss = xent(logits, targets)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/(idx+1),
                     num_correct/batch, total_correct/total))

In [5]:
def training(epoch, model, dataloader, optimizer, num_classes, prox, conprox,
             optim_prox, optim_conprox, weight_prox, weight_conprox,
             epsilon=8/255, alpha=2/255, num_repeats=10, use_at=True):
    model.train()
    total = 0
    total_loss = 0
    total_correct = 0
    xent = nn.CrossEntropyLoss()
    for idx, (inputs, targets) in enumerate(dataloader):
        inputs, targets = inputs.cuda(), targets.cuda()
        batch = inputs.size(0)
        
        if use_at:
            x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
        else:
            x = inputs
        logits, features = model(x)
        
        loss_xent = xent(logits, targets)
        loss_prox = prox(features, targets) * weight_prox
        loss_conprox = conprox(features, targets) * weight_conprox
        loss = loss_xent + loss_prox - loss_conprox
        
        optimizer.zero_grad()
        optim_prox.zero_grad()
        optim_conprox.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update trainable centroid of center loss
        for param in prox.parameters():
            param.grad.data *= 1/weight_prox
        optim_prox.step()
        
        # Update trainable centroid of pc loss
        for param in conprox.parameters():
            param.grad.data *= 1/weight_conprox
        optim_conprox.step()
            
        total += batch
        total_loss += loss.item()
        num_correct = torch.argmax(logits.data, dim=1).eq(targets.data).cpu().sum().item()
        total_correct += num_correct
        
        if idx % 100 == 0:
            print('Epoch %d [%d/%d] | loss: %.4f (avg: %.4f) | acc: %.4f (avg: %.4f) |'\
                  % (epoch, idx, len(dataloader), loss.item(), total_loss/(idx+1),
                     num_correct/batch, total_correct/total))

In [6]:
def evaluation(epoch, model, dataloader, alpha, epsilon, num_repeats):
    model.eval()
    total_correct_nat = 0
    total_correct_adv = 0
    
    xent = nn.CrossEntropyLoss()
    with tqdm(dataloader) as pbar:
        for index, (inputs, targets) in enumerate(pbar):
            inputs, targets = inputs.cuda(), targets.cuda()
            batch = inputs.size(0)
            pbar.set_description("[eval] %d/%d" % (index+1, len(dataloader.dataset)))
            
            x = inner_max(model, xent, inputs, targets, epsilon, alpha, num_repeats)
            with torch.no_grad():
                logits_nat = model(inputs)[0]
                logits_adv = model(x)[0]
            total_correct_nat += torch.argmax(logits_nat.data, dim=1).eq(targets.data).cpu().sum().item()
            total_correct_adv += torch.argmax(logits_adv.data, dim=1).eq(targets.data).cpu().sum().item()
            pbar.set_postfix(OrderedDict(acc_nat=total_correct_nat / len(dataloader.dataset),
                                         acc_rob=total_correct_adv / len(dataloader.dataset)))
    
    return (total_correct_nat / len(dataloader.dataset)), (total_correct_adv / len(dataloader.dataset))

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = gpu
os.makedirs(checkpoint, exist_ok=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()])
train_dataset, _ = get_dataloader(dataset, batch_size)
num_samples = len(train_dataset)
num_samples_for_train = int(num_samples * 0.98)
num_samples_for_valid = num_samples - num_samples_for_train
train_set, valid_set = random_split(train_dataset, [num_samples_for_train, num_samples_for_valid])
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_set, batch_size=1, shuffle=True, drop_last=False)

model = nn.DataParallel(get_network(model_type, num_classes, use_pc=True).cuda())
optimizer = optim.SGD(model.parameters(),lr=lr, momentum=momentum)

for epoch in range(warmup_epochs):
    xent_training(epoch, model, train_dataloader, optimizer, num_classes)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)

num_hidden = model.module.fc2.in_features
prox = Proximity(num_classes=num_classes, feat_dim=num_hidden, use_gpu=True)
conprox = Con_Proximity(num_classes=num_classes, feat_dim=num_hidden, use_gpu=True)
optim_prox = optim.SGD(prox.parameters(), lr=lr_prox)
optim_conprox = optim.SGD(conprox.parameters(), lr=lr_conprox)

scheduler = [200, 250]
adjust_learning_rate = lr_scheduler.MultiStepLR(optimizer, scheduler, gamma=0.1)
adjust_learning_rate_prox = lr_scheduler.MultiStepLR(optim_prox, scheduler, gamma=0.1)
adjust_learning_rate_conprox = lr_scheduler.MultiStepLR(optim_conprox, scheduler, gamma=0.1)
best_acc_nat, best_acc_rob = 0, 0

for epoch in range(with_pc_epochs):
    training(epoch, model, train_dataloader, optimizer, num_classes, prox, conprox,
             optim_prox, optim_conprox, weight_prox, weight_conprox, epsilon, alpha, num_repeats)
    test_acc_nat, test_acc_rob = evaluation(epoch, model, valid_dataloader, alpha, epsilon, num_repeats)
        
    is_best = best_acc_nat < test_acc_nat and best_acc_rob < test_acc_rob
    best_acc_nat = max(best_acc_nat, test_acc_nat)
    best_acc_rob = max(best_acc_rob, test_acc_rob)
    save_checkpoint = {'state_dict': model.state_dict(),
                       'best_acc_nat': best_acc_nat,
                       'best_acc_rob': best_acc_rob,
                       'optimizer': optimizer.state_dict(),
                       'model_type': model_type,
                       'dataset': dataset}
    torch.save(save_checkpoint, os.path.join(checkpoint, 'model'))
    if is_best:
        torch.save(save_checkpoint, os.path.join(checkpoint, 'best_model'))
    adjust_learning_rate.step()
    adjust_learning_rate_prox.step()
    adjust_learning_rate_conprox.step()

Files already downloaded and verified
Files already downloaded and verified
Epoch 0 [0/192] | loss: 2.6742 (avg: 2.6742) | acc: 0.0000 (avg: 0.0000) |
Epoch 0 [100/192] | loss: 2.1932 (avg: 2.3182) | acc: 0.1641 (avg: 0.1124) |


[eval] 1000/1000: 100%|██████████| 1000/1000 [03:28<00:00,  4.79it/s, acc_nat=0.256, acc_rob=0.204]


Epoch 1 [0/192] | loss: 2.1504 (avg: 2.1504) | acc: 0.1914 (avg: 0.1914) |
Epoch 1 [100/192] | loss: 2.1140 (avg: 2.1235) | acc: 0.2227 (avg: 0.2093) |


[eval] 581/1000:  58%|█████▊    | 580/1000 [02:02<01:26,  4.83it/s, acc_nat=0.182, acc_rob=0.135]

Epoch 2 [100/192] | loss: 1.9669 (avg: 2.0513) | acc: 0.2500 (avg: 0.2350) |


...

[eval] 1000/1000: 100%|██████████| 1000/1000 [03:18<00:00,  5.05it/s, acc_nat=0.811, acc_rob=0.421]


Epoch 198 [0/192] | loss: 0.4464 (avg: 0.4464) | acc: 0.8281 (avg: 0.8281) |
Epoch 198 [100/192] | loss: 0.4713 (avg: 0.4682) | acc: 0.8203 (avg: 0.8361) |


[eval] 326/1000:  32%|███▎      | 325/1000 [01:05<02:14,  5.03it/s, acc_nat=0.27, acc_rob=0.147] 

Epoch 201 [100/192] | loss: 0.3166 (avg: 0.3477) | acc: 0.9023 (avg: 0.8830) |


[eval] 297/1000:  30%|██▉       | 296/1000 [00:59<02:18,  5.07it/s, acc_nat=0.253, acc_rob=0.126]